In [3]:
!pip install torch transformers accelerate

In [4]:
from huggingface_hub import login
login()  # will prompt you to paste your HF token

In [5]:
import time, torch
from transformers import AutoTokenizer, AutoModelForCausalLM

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device → {device}")

Using device → cuda


In [8]:
# TARGET_CKPT = "meta-llama/Llama-2-7b-hf"
# ASSIST_CKPT = "meta-llama/Llama-2-70m-hf"
# Set these to open‑access models:
TARGET_CKPT  = "EleutherAI/gpt-neo-2.7B"
ASSIST_CKPT  = "EleutherAI/gpt-neo-125M"

In [9]:
print("Loading tokenizer & models… this may take a minute")
tokenizer = AutoTokenizer.from_pretrained(TARGET_CKPT)

try:
    target_model = AutoModelForCausalLM.from_pretrained(
        TARGET_CKPT, torch_dtype=torch.float16, device_map="auto"
    )
    assistant_model = AutoModelForCausalLM.from_pretrained(
        ASSIST_CKPT, torch_dtype=torch.float16, device_map="auto"
    )
except Exception as e:
    print("FP16 load failed, falling back to 8‑bit:", e)
    target_model = AutoModelForCausalLM.from_pretrained(
        TARGET_CKPT, load_in_8bit=True, device_map="auto"
    )
    assistant_model = AutoModelForCausalLM.from_pretrained(
        ASSIST_CKPT, load_in_8bit=True, device_map="auto"
    )

Loading tokenizer & models… this may take a minute


tokenizer_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/10.7G [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/526M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

In [14]:
prompt = "In a distant future, trump will "
inputs = tokenizer(prompt, return_tensors="pt").to(device)

In [15]:
start = time.time()
std_ids = target_model.generate(
    **inputs,
    max_new_tokens=50,
    do_sample=False
)
std_time = time.time() - start
std_text = tokenizer.decode(std_ids[0], skip_special_tokens=True)

print(f"\n⏱️ Standard decoding: {std_time:.2f}s")
print(std_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



⏱️ Standard decoding: 1.92s
In a distant future, trump will 
be the only superpower.
And the only superpower will be 
the United States.
And the United States will be 
the only superpower.
And the only superpower will be 
the United States.
And the United States


In [16]:
start = time.time()
spec_ids = target_model.generate(
    **inputs,
    max_new_tokens   = 50,
    assistant_model  = assistant_model,  # ← enables spec‑decoding
    do_sample        = False             # greedy; set True to sample
)
spec_time = time.time() - start
spec_text = tokenizer.decode(spec_ids[0], skip_special_tokens=True)

print(f"\n⏱️ Speculative decoding: {spec_time:.2f}s")
print(spec_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene


⏱️ Speculative decoding: 1.48s
In a distant future, trump will 
be the only superpower.
And the only superpower will be 
the United States.
And the United States will be 
the only superpower.
And the only superpower will be 
the United States.
And the United States


In [17]:
print(f"\n🚀 Speed‑up: {std_time/spec_time:.2f}×")


🚀 Speed‑up: 1.30×
